## Spam the scammers

This code spams a fraudulent contact form of a scammer with false credit card details so that it becomes impossible to distinguish real from fake people and so that less harm is done!

In [ ]:
# install required dependencies
%pip install selenium
%pip install Faker
%pip install beautifulsoup4
%pip install requests
%pip install pandas

### Import dependencies

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.common.exceptions import WebDriverException, TimeoutException

from faker import Faker
import random

import requests
from bs4 import BeautifulSoup

import pandas as pd

### Get a list of free proxy servers

After a certain number of (automated) form submissions, you'll get blocked by the webserver. Thus, we need to connect through a proxy which will hide our IP address so that we can spam a fraudulent website with fake data indefinitly (we just need to change our IP address after it got blocked)

In [ ]:
def get_proxies():
    # Getting proxy list from website
    proxy_list = requests.get('https://free-proxy-list.net/')

    # Parsing the HTML code
    soup = BeautifulSoup(proxy_list.content, 'html.parser')

    # Find table with all proxy addresses
    table = soup.find_all('table')

    # get all rows in the table
    rows = table[0].find('tbody').find_all('tr')

    # loop through all rows and store IP addresses and ports if proxy is anonymous
    ip_address_list = list()
    port_list = list()
    for row in rows:
        columns = row.find_all('td')
        ip_address = columns[0].text
        port = columns[1].text
        security = columns[4].text

        # elite proxy server are anonymous
        if security.find('elite') != -1:
            ip_address_list.append(ip_address)
            port_list.append(port)

    df = pd.DataFrame({'ip_address': ip_address_list, 'port': port_list})
    return df

### Setup webbrowser

We have to select a web browser, ideally one that we don't use on a daily basis. We set up a new profile for firefox here, which includes the ip address and port we got from the proxy server.

In [ ]:
def get_driver(ip, port):
    # Create Firefox profile and set proxy settings
    profile = FirefoxProfile()
    profile.set_preference("network.proxy.type", 1)
    profile.set_preference("network.proxy.http", ip)
    profile.set_preference("network.proxy.http_port", int(port))
    profile.set_preference("network.proxy.ssl", ip)
    profile.set_preference("network.proxy.ssl_port", int(port))
    profile.set_preference("network.proxy.share_proxy_settings", True)
    profile.set_preference("network.proxy.no_proxies_on", "")  # Proxy everything
    profile.update_preferences()

    # Create options object and attach profile
    options = Options()
    options.profile = profile

    # Start the WebDriver
    return webdriver.Firefox(options=options)

### Main loop

Here, we first get a list of proxy servers and then cycle through them as long as the IP addresses are working. We test in a try/except block if the IP address is working (i.e. it hasn't been blocked by the webserver yet). If so, we get the fraudulent website, and enter fake data automatically and submit the form. If we reach the end of all IP addresses, we get a new list of fresh IP addresses and start from the beginning. This script can be left running indefinitly, until the fraudulent website is eventually taken down.

In [ ]:
# get list of proxies
proxies = get_proxies()

# set initial ip address and port
ip = proxies['ip_address'][0]
port = proxies['port'][0]

# proxy counter
proxy_counter = 1

# get initial driver, hiding ip address through proxy
driver = get_driver(ip, port)
driver.set_page_load_timeout(30)

# loop indefinitly
fake_data_counter = 0
while True:
    try:
        # Navigate to the webpage with the form
        driver.get('https://namida-tokyo.com/a2/login.php')

        # get input fields and submit button
        email_field = driver.find_element(By.ID, 'inputEmail')
        password_field = driver.find_element(By.ID, 'inputPassword')
        login_button = driver.find_element(By.ID, 'login')

        # generate random first and last name
        first_name = Faker('en_US').first_name()
        last_name = Faker('en_US').last_name()
        domain = random.choice(['gmail.com', 'yahoo.com', 'hotmail.com', 'outlook.com'])
        email = first_name.lower() + '.' + last_name.lower() + '@' + domain

        # generate fake password
        password = Faker('en_US').password()

        # fill in form
        email_field.send_keys(email)
        password_field.send_keys(password)
        login_button.click()

        # next page should load, now asking for credit card details
        cardholder_name_field = driver.find_element(By.ID, 'cardholdername')
        card_number_field = driver.find_element(By.ID, 'card_number')
        card_expiration_field = driver.find_element(By.ID, 'card_expiry')
        card_cvc_field = driver.find_element(By.ID, 'cvc_code')
        submit_button = driver.find_element(By.ID, 'submit-payment')

        # generate fake credit card number
        card_number = Faker('en_US').credit_card_number()
        card_expiration = Faker('en_US').credit_card_expire()
        card_cvc = Faker('en_US').credit_card_security_code()

        # fill in form
        cardholder_name_field.send_keys(first_name + ' ' + last_name)
        card_number_field.send_keys(card_number)
        card_expiration_field.send_keys(card_expiration)
        card_cvc_field.send_keys(card_cvc)
        submit_button.click()

        # increment fake data counter
        fake_data_counter += 1
        print(f'Fake data submitted: {fake_data_counter}')
    except (WebDriverException, TimeoutException) as e:
        print(f"Failed with {ip}:{port} — {e}")
        print('getting new ip and port and I\'ll try again ...')
        
        if proxy_counter == len(proxies):
            proxies = get_proxies()
            proxy_counter = 0
        
        ip = proxies['ip_address'][proxy_counter]
        port = proxies['port'][proxy_counter]            

        driver.quit()
        driver = get_driver(ip, port)
        driver.set_page_load_timeout(30)

        proxy_counter += 1

        continue
